# Run workflow using API

In [4]:
from pathlib import Path

from rich import print as pprint

In [5]:
import logging

logging.basicConfig(level=logging.WARNING)
# Set to WARNING to see only warnings
# Set to INFO to see sparql queries
# Set to DEBUG to see raw results

In [6]:
# Silence dask logging
!dask config set logging.distributed warning

Updated [logging.distributed] to [warning], config saved to /home/stefanv/.config/dask/dask.yaml


## Setup session directory

In this directory files like structure files are stored and a DuckDB database for meta data.

In [7]:
session_dir = Path("session1")
session_dir

PosixPath('session1')

## Search Uniprot for structures

In [4]:
from protein_quest.uniprot import Query

from protein_detective.workflow import search_structures_in_uniprot

In [5]:
query = Query(
    taxon_id="9606",
    reviewed=True,
    subcellular_location_uniprot="nucleus",
    subcellular_location_go=["GO:0005634"],  # Cellular component - Nucleus
    molecular_function_go=["GO:0003677"],  # Molecular function - DNA binding
)

In [6]:
nr_uniprot, nr_pdbe, nr_pdb_uniprot, nr_af = search_structures_in_uniprot(query, session_dir, limit=100)
nr_uniprot, nr_pdbe, nr_pdb_uniprot, nr_af

(100, 100, 100, 100)

Use [database queries](#query-session-database) to see what was found.

## Fetch structures from PDBe and Alphafold found in Uniprot


In [7]:
from protein_detective.workflow import async_retrieve_structures

In [8]:
# Allow asyncio to be nested (needed for jupyter notebooks)
download_path, nr_pdbs, nr_alphafolds = await async_retrieve_structures(session_dir)
download_path, nr_pdbs, nr_alphafolds

Fetching Alphafold summaries: 100%|██████████| 100/100 [00:03<00:00, 30.30it/s]


(PosixPath('session1/downloads'), 100, 100)

## Filter structures

Prepare structures for powerfitting by filtering them based on confidence and nr of residues.

In [8]:
from protein_detective.filter import ConfidenceFilterQuery, FilterOptions, SecondaryStructureFilterQuery
from protein_detective.workflow import filter_structures

In [9]:
options = FilterOptions(
    confidence=ConfidenceFilterQuery(confidence=70, min_residues=100, max_residues=1000),
    secondary_structure=SecondaryStructureFilterQuery(),
)

In [10]:
filtered_dir, filtered_results = filter_structures(session_dir, options)
pprint(filtered_results[:2])
pprint(filtered_results[-2:])
filtered_dir

  0%|          | 0/100 [00:00<?, ?file/s]

[
    FilteredStructure(
        uniprot_accession='A0A087WUV0',
        pdb_id=None,
        confidence=ConfidenceFilterResult(
            input_file='AF-A0A087WUV0-F1-model_v4.cif',
            count=283,
            filtered_file=PosixPath('filtered/AF-A0A087WUV0-F1-model_v4.cif')
        ),
        chain=None,
        residue=None,
        secondary_structure=None
    ),
    FilteredStructure(
        uniprot_accession='A0A0C5B5G6',
        pdb_id=None,
        confidence=ConfidenceFilterResult(
            input_file='AF-A0A0C5B5G6-F1-model_v4.cif',
            count=10,
            filtered_file=None
        ),
        chain=None,
        residue=None,
        secondary_structure=None
    )
]

[
    FilteredStructure(
        uniprot_accession='O14646',
        pdb_id='4NW2',
        confidence=None,
        chain=ChainFilterStatistics(
            input_file=PosixPath('session1/downloads/pdbe/4nw2.cif.gz'),
            chain_id='A',
            passed=True,
            output_file=PosixPath('pdb_chain_filtered/4nw2_A2A.cif.gz'),
            discard_reason=None
        ),
        residue=ResidueFilterStatistics(
            input_file=PosixPath('session1/pdb_chain_filtered/4nw2_A2A.cif.gz'),
            residue_count=173,
            passed=True,
            output_file=PosixPath('filtered/4nw2_A2A.cif')
        ),
        secondary_structure=None
    ),
    FilteredStructure(
        uniprot_accession='O00257',
        pdb_id='3I8Z',
        confidence=None,
        chain=ChainFilterStatistics(
            input_file=PosixPath('session1/downloads/pdbe/3i8z.cif.gz'),
            chain_id='A',
            passed=True,
            output_file=PosixPath('pdb_chain_filtered/3i8z_A2A.cif.gz'),
            discard_reason=None
        ),
        residue=ResidueFilterStatistics(
            input_file=PosixPath('session1/pdb_chain_filtered/3i8z_A2A.cif.gz'),
            residue_count=109,
            passed=True,
            output_file=PosixPath('filtered/3i8z_A2A.cif')
        ),
        secondary_structure=None
    )
]

PosixPath('session1/filtered')

In [12]:
total_nr = len(filtered_results)
total_nr_passed = sum(1 for r in filtered_results if r.passed)
total_nr_discarded = total_nr - total_nr_passed
pprint(f"Total entries: {total_nr}, passed: {total_nr_passed}, discarded: {total_nr_discarded}")

Total entries: 200, passed: 132, discarded: 68

## Powerfit


In [13]:
from protein_detective.powerfit.options import PowerfitOptions
from protein_detective.powerfit.workflow import powerfit_commands

To run you must have cloned the https://github.com/haddocking/powerfit-tutorial repository in '../../powerfit-tutorial'.

In [14]:
options = PowerfitOptions(
    target=Path("../../powerfit-tutorial/ribosome-KsgA.map"),
    resolution=13,
    angle=20,
    laplace=True,
)

### Run

Here we use printed commands to run powerfit, see [powerfit.ipynb](powerfit.ipynb) for running powerfits using the API.

In [15]:
commands, powerfit_run_id = powerfit_commands(session_dir, options)

In [16]:
powerfit_run_id

1

In [17]:
cwd = str(Path.cwd())
rel_commands = [c.replace(cwd, ".") for c in commands]
pprint(rel_commands[:10])

[
    'powerfit ./session1/powerfit/1/ribosome-KsgA.map 13 ./session1/filtered/AF-A0A087WUV0-F1-model_v4.cif 
--laplace --resampling-rate 2 --num 0 --nproc 1 --directory ./session1/powerfit/1/AF-A0A087WUV0-F1-model_v4 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/1/ribosome-KsgA.map 13 ./session1/filtered/AF-A0A0U1RQI7-F1-model_v4.cif 
--laplace --resampling-rate 2 --num 0 --nproc 1 --directory ./session1/powerfit/1/AF-A0A0U1RQI7-F1-model_v4 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/1/ribosome-KsgA.map 13 ./session1/filtered/AF-A0A1B0GTS1-F1-model_v4.cif 
--laplace --resampling-rate 2 --num 0 --nproc 1 --directory ./session1/powerfit/1/AF-A0A1B0GTS1-F1-model_v4 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/1/ribosome-KsgA.map 13 ./session1/filtered/AF-A0A1B0GWH4-F1-model_v4.cif 
--laplace --resampling-rate 2 --num 0 --nproc 1 --directory ./session1/powerfit/1/AF-A0A1B0GWH4-F1-model_v4 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/1/ribosome-KsgA.map 13 ./session1/filtered/AF-A0A1W2PPF3-F1-model_v4.cif 
--laplace --resampling-rate 2 --num 0 --nproc 1 --directory ./session1/powerfit/1/AF-A0A1W2PPF3-F1-model_v4 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/1/ribosome-KsgA.map 13 ./session1/filtered/AF-A0A1W2PQL4-F1-model_v4.cif 
--laplace --resampling-rate 2 --num 0 --nproc 1 --directory ./session1/powerfit/1/AF-A0A1W2PQL4-F1-model_v4 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/1/ribosome-KsgA.map 13 ./session1/filtered/AF-A1YPR0-F1-model_v4.cif --laplace 
--resampling-rate 2 --num 0 --nproc 1 --directory ./session1/powerfit/1/AF-A1YPR0-F1-model_v4 --delimiter , --angle
20',
    'powerfit ./session1/powerfit/1/ribosome-KsgA.map 13 ./session1/filtered/AF-A2RRD8-F1-model_v4.cif --laplace 
--resampling-rate 2 --num 0 --nproc 1 --directory ./session1/powerfit/1/AF-A2RRD8-F1-model_v4 --delimiter , --angle
20',
    'powerfit ./session1/powerfit/1/ribosome-KsgA.map 13 ./session1/filtered/AF-A3KN83-F1-model_v4.cif --laplace 
--resampling-rate 2 --num 0 --nproc 1 --directory ./session1/powerfit/1/AF-A3KN83-F1-model_v4 --delimiter , --angle
20',
    'powerfit ./session1/powerfit/1/ribosome-KsgA.map 13 ./session1/filtered/AF-A6NDX5-F1-model_v4.cif --laplace 
--resampling-rate 2 --num 0 --nproc 1 --directory ./session1/powerfit/1/AF-A6NDX5-F1-model_v4 --delimiter , --angle
20'
]

These commands should be run on a cluster. Here we will just run a couple of them to show how it works.

In [18]:
!{rel_commands[6]}

Target file read from:                                                          
/home/stefanv/git/protein-detective/protein-detective/docs/session1/powerfit/1/r
ibosome-KsgA.map                                                                
Target resolution: 13.00                                                        
Initial shape of density: 128 128 128                                           
Shape after trimming: 60 73 67                                                  
Shape after extending: 60 75 70                                                 
Template file read from:                                                        
/home/stefanv/git/protein-detective/protein-detective/docs/session1/filtered/AF-
A1YPR0-F1-model_v4.cif                                                          
Reading in rotations.                                                           
Requested rotational sampling density: 20.00                                    
Real rotational sampling den

In [19]:
!{rel_commands[0]}

Target file read from:                                                          
/home/stefanv/git/protein-detective/protein-detective/docs/session1/powerfit/1/r
ibosome-KsgA.map                                                                
Target resolution: 13.00                                                        
Initial shape of density: 128 128 128                                           
Shape after trimming: 60 73 67                                                  
Shape after extending: 60 75 70                                                 
Template file read from:                                                        
/home/stefanv/git/protein-detective/protein-detective/docs/session1/filtered/AF-
A0A087WUV0-F1-model_v4.cif                                                      
Reading in rotations.                                                           
Requested rotational sampling density: 20.00                                    
Real rotational sampling den

In [20]:
!{rel_commands[-1]}

Target file read from:                                                          
/home/stefanv/git/protein-detective/protein-detective/docs/session1/powerfit/1/r
ibosome-KsgA.map                                                                
Target resolution: 13.00                                                        
Initial shape of density: 128 128 128                                           
Shape after trimming: 60 73 67                                                  
Shape after extending: 60 75 70                                                 
Template file read from:                                                        
/home/stefanv/git/protein-detective/protein-detective/docs/session1/filtered/5db
2_A2A.cif                                                                       
Reading in rotations.                                                           
Requested rotational sampling density: 20.00                                    
Real rotational sampling den

See [powerfit.ipynb](powerfit.ipynb) for running all powerfits using the API.

### Report

Once all powerfit jobs are done the results can be parsed and reported.


In [21]:
from protein_detective.powerfit.workflow import powerfit_report

In [22]:
solutions = powerfit_report(session_dir)

In [23]:
len(solutions)

4812

In [24]:
solutions

,powerfit_run_id,structure,rank,cc,fishz,relz,translation,rotation,pdb_file,uniprot_acc,pdb_id
0,1,AF-A1YPR0-F1-model_v4,1,0.280,0.288,11.087,"[162.71, 132.01, 181.13]","[-1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0]",session1/filtered/AF-A1YPR0-F1-model_v4.cif,A1YPR0,None
1,1,AF-A1YPR0-F1-model_v4,2,0.279,0.287,11.049,"[156.57, 156.57, 135.08]","[-0.548, 0.184, 0.816, -0.548, -0.816, -0.184,...",session1/filtered/AF-A1YPR0-F1-model_v4.cif,A1YPR0,None
2,1,AF-A1YPR0-F1-model_v4,3,0.275,0.282,10.870,"[165.78, 153.5, 141.22]","[-0.632, 0.548, 0.548, -0.548, -0.816, 0.184, ...",session1/filtered/AF-A1YPR0-F1-model_v4.cif,A1YPR0,None
3,1,AF-A1YPR0-F1-model_v4,4,0.266,0.273,10.518,"[257.88, 270.16, 178.06]","[0.0, -0.797, -0.604, 0.0, -0.604, 0.797, -1.0...",session1/filtered/AF-A1YPR0-F1-model_v4.cif,A1YPR0,None
4,1,AF-A1YPR0-F1-model_v4,5,0.265,0.272,10.484,"[141.22, 159.64, 205.69]","[0.548, -0.816, -0.184, -0.548, -0.184, -0.816...",session1/filtered/AF-A1YPR0-F1-model_v4.cif,A1YPR0,None
...,...,...,...,...,...,...,...,...,...,...,...
4807,1,5db2_A2A,1814,0.081,0.081,4.472,"[233.32, 202.62, 159.64]","[-0.548, -0.548, -0.632, 0.184, -0.816, 0.548,...",session1/filtered/5db2_A2A.cif,O00255,5DB2
4808,1,5db2_A2A,1813,0.081,0.081,4.475,"[150.43, 168.85, 141.22]","[0.258, -0.362, 0.896, 0.362, 0.896, 0.258, -0...",session1/filtered/5db2_A2A.cif,O00255,5DB2
4809,1,5db2_A2A,1812,0.081,0.081,4.475,"[125.87, 168.85, 214.9]","[-0.362, 0.258, 0.896, -0.258, 0.896, -0.362, ...",session1/filtered/5db2_A2A.cif,O00255,5DB2
4810,1,5db2_A2A,1811,0.081,0.081,4.488,"[125.87, 162.71, 217.97]","[-0.632, 0.548, 0.548, 0.548, -0.184, 0.816, 0...",session1/filtered/5db2_A2A.cif,O00255,5DB2


### Fit model to solution

Rotate/translate the input model PDB files to the top 5 powerfit solutions.

In [14]:
from protein_detective.powerfit.workflow import powerfit_fit_models

In [15]:
fitted = powerfit_fit_models(session_dir, top=5)
fitted

Writing fitted model PDB files: 100%|██████████| 5/5 [00:00<00:00, 90.13it/s]


,powerfit_run_id,structure,rank,fitted_model_file,unfitted_model_file
index,,,,,
0,1,AF-A1YPR0-F1-model_v4,1,session1/powerfit/1/AF-A1YPR0-F1-model_v4/fit_...,session1/filtered/AF-A1YPR0-F1-model_v4.cif
1,1,AF-A1YPR0-F1-model_v4,1,session1/powerfit/1/AF-A1YPR0-F1-model_v4/fit_...,session1/filtered/AF-A1YPR0-F1-model_v4.cif
2,1,AF-A1YPR0-F1-model_v4,1,session1/powerfit/1/AF-A1YPR0-F1-model_v4/fit_...,session1/filtered/AF-A1YPR0-F1-model_v4.cif
3,1,AF-A1YPR0-F1-model_v4,1,session1/powerfit/1/AF-A1YPR0-F1-model_v4/fit_...,session1/filtered/AF-A1YPR0-F1-model_v4.cif
4,1,AF-A1YPR0-F1-model_v4,1,session1/powerfit/1/AF-A1YPR0-F1-model_v4/fit_...,session1/filtered/AF-A1YPR0-F1-model_v4.cif
